In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("example-pyspark-read-and-write").getOrCreate()

In [2]:
HDFS_HOST="hdfs://namenode:9000/"
# Create data
data = [('First', 1), ('Second', 2), ('Third', 3), ('Fourth', 4), ('Fifth', 5)]
df = sparkSession.createDataFrame(data)

# Write into HDFS
df.write.csv(HDFS_HOST+"test/example.csv")
# để kiểm tra vào shell của name node 
# dùng lệnh hdfs dfs -ls /
# sẽ hiển thị thông tin file vừa ghi

In [2]:
from pyspark.sql.types import *

schema = StructType([   
        StructField("time", StringType(), True),
        StructField("text", StringType(), True),
        StructField("retweet_count", DoubleType(), True),
        StructField("location", StringType(), True),
        StructField("favorite_count", DoubleType(), True),
        StructField("user_id", StringType(), True),
        StructField("place", StringType(), True),
        StructField("user_followers_count", StringType(), True),
        StructField("sentiment", StringType(), True)
    
])

In [3]:
csvDF = spark \
    .readStream \
    .schema(schema) \
    .csv("hdfs://namenode:9000/data/biden.csv/")

In [4]:
query = csvDF.writeStream.queryName("device_counts").format("memory")\
    .start()

In [6]:
spark.sql('SELECT time FROM device_counts').show(20, False)

+----------------------------------------+
|time                                    |
+----------------------------------------+
|Mon Dec 21 12:41:25 +0000 2020          |
|Mon Dec 21 12:41:26 +0000 2020          |
|Mon Dec 21 12:41:27 +0000 2020          |
|Mon Dec 21 12:41:28 +0000 2020          |
|Mon Dec 21 12:41:28 +0000 2020          |
|Mon Dec 21 12:41:29 +0000 2020          |
|Mon Dec 21 12:41:29 +0000 2020          |
|Mon Dec 21 12:41:30 +0000 2020          |
|#BidenCrimeFamilly                      |
|Trump won in a landslide!"              |
|Mon Dec 21 12:41:32 +0000 2020          |
|Mon Dec 21 12:41:33 +0000 2020          |
|Mon Dec 21 12:41:33 +0000 2020          |
|Mon Dec 21 12:41:33 +0000 2020          |
|Mon Dec 21 12:41:33 +0000 2020          |
|#bidenwonDonnieLost #bidenwonDonaldlost"|
|Mon Dec 21 12:41:33 +0000 2020          |
|Mon Dec 21 12:41:33 +0000 2020          |
|Mon Dec 21 12:41:34 +0000 2020          |
|Mon Dec 21 12:41:34 +0000 2020          |
+----------